<p><font size="6"><b>Creating maps with Cartopy</b></font></p>


> *DS Python for GIS and Geoscience*  
> *September, 2023*
>
> *© 2023, Joris Van den Bossche and Stijn Van Hoey. Licensed under [CC BY-SA 4.0 Creative Commons](https://creativecommons.org/licenses/by-sa/4.0/)* Adapted from material from Phil Elson and Ryan Abernathey (see below).

> *Adapted in 2024 for the course 'Computational Physics' by Wout Dewettinck*
---

**Table of contents**<a id='toc0_'></a>    
- [Maps in Scientific Python](#toc1_)    
    - [Credit: Phil Elson and Ryan Abernathey](#toc1_1_1_)    
  - [Background: Projections](#toc1_2_)    
    - [Most of our media for visualization *are* flat](#toc1_2_1_)    
    - [[Map] Projections: Taking us from spherical to flat](#toc1_2_2_)    
    - [The major problem with map projections](#toc1_2_3_)    
  - [Introducing Cartopy](#toc1_3_)    
    - [Cartopy Projections and other reference systems](#toc1_3_1_)    
    - [Drawing a map](#toc1_3_2_)    
    - [Useful methods of a GeoAxes](#toc1_3_3_)    
    - [Some More Examples of Different Global Projections](#toc1_3_4_)    
    - [Regional Maps](#toc1_3_5_)    
  - [Adding features to the map](#toc1_4_)    
  - [Adding vector data to the map](#toc1_5_)    
  - [Adding raster data to the map](#toc1_6_)    
  - [Xarray Integration](#toc1_7_)    
  - [Doing More](#toc1_8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
import xarray as xr
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'png' # Otherwise the figures take too long to load + the notebooks crash sometimes

In [ ]:
data_dir = "/data/gent/452/vsc45263/computphys/data"  # "/data/gent/courses/2024/atmmod_C002867/input"


# <a id='toc1_'></a>[Maps in Scientific Python](#toc0_)

Making maps is a fundamental part of geoscience research.
Maps differ from regular figures in the following principle ways:

- Maps require a *projection* of geographic coordinates on the 3D Earth to the 2D space of your figure.
- Maps often include extra decorations besides just our data (e.g. continents, country borders, etc.)

In this notebook, we will learn about [Cartopy](https://scitools.org.uk/cartopy/docs/latest/), one of the most common packages for making maps within python.

### <a id='toc1_1_1_'></a>[Credit: Phil Elson and Ryan Abernathey](#toc0_)

Lots of the material in this notebook was adopted from https://earth-env-data-science.github.io/intro.html by Ryan Abernathey, which itself was adopted from [Phil Elson](https://pelson.github.io/)'s excellent [Cartopy Tutorial](https://github.com/SciTools/cartopy-tutorial). Phil is the creator of Cartopy and published his tutorial under an [open license](http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/), meaning that we can copy, adapt, and redistribute it as long as we give proper attribution.

## <a id='toc1_2_'></a>[Background: Projections](#toc0_)

### <a id='toc1_2_1_'></a>[Most of our media for visualization *are* flat](#toc0_)

Our two most common media are flat:

 * Paper
 * Screen

![](https://raw.githubusercontent.com/SciTools/cartopy-tutorial/master/static/flat_medium.jpg)

### <a id='toc1_2_2_'></a>[[Map] Projections: Taking us from spherical to flat](#toc0_)

A map projection (or more commonly refered to as just "projection") is:

> a systematic transformation of the latitudes and longitudes of locations from the surface of a sphere or an ellipsoid into locations on a plane. [[Wikipedia: Map projection](https://en.wikipedia.org/wiki/Map_projection)].

### <a id='toc1_2_3_'></a>[The major problem with map projections](#toc0_)

![orange peel](https://raw.githubusercontent.com/SciTools/cartopy-tutorial/master/static/orange_peel.jpg)

 * The surface of a sphere is topologically different to a 2D surface, therefore we *have* to cut the sphere *somewhere*
 * A sphere's surface cannot be represented on a plane without distortion.
 
There are many different ways to make a projection, and we will not attempt to explain all of the choices and tradeoffs here. Instead, you can read Phil's [original tutorial](https://github.com/SciTools/cartopy-tutorial/blob/master/tutorial/projections_crs_and_terms.ipynb) for a great overview of this topic.
Instead, we will dive into the more practical sides of Caropy usage.

## <a id='toc1_3_'></a>[Introducing Cartopy](#toc0_)

https://scitools.org.uk/cartopy/docs/latest/

Cartopy makes use of the powerful [PROJ.4](https://proj4.org/), numpy and shapely libraries and includes a programatic interface built on top of Matplotlib for the creation of publication quality maps.

Key features of cartopy are its object oriented projection definitions, and its ability to transform points, lines, vectors, polygons and images between those projections.

### <a id='toc1_3_1_'></a>[Cartopy Projections and other reference systems](#toc0_)

In Cartopy, each projection is a class.
Most classes of projection can be configured in projection-specific ways, although Cartopy takes an opinionated stance on sensible defaults.

Let's create a Plate Carree projection instance.

To do so, we need cartopy's crs module. This is typically imported as ``ccrs`` (Cartopy Coordinate Reference Systems).

Cartopy's projection list tells us that the Plate Carree projection is available with the ``ccrs.PlateCarree`` class: https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#cartopy-projections 

**Note:** we need to *instantiate* the class in order to do anything projection-y with it!

In [ ]:
ccrs.PlateCarree()

### <a id='toc1_3_2_'></a>[Drawing a map](#toc0_)

Cartopy optionally depends upon matplotlib, and each projection knows how to create a matplotlib Axes (or AxesSubplot) that can represent itself.

The Axes that the projection creates is a [cartopy.mpl.geoaxes.GeoAxes](https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.mpl.geoaxes.GeoAxes.html). This Axes subclass overrides some of matplotlib's existing methods, and adds a number of extremely useful ones for drawing maps:

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax

That was a little underwhelming, but we can see that the Axes created is indeed one of those GeoAxes[Subplot] instances.

One of the most useful methods that this class adds on top of the standard matplotlib Axes class is the ``coastlines`` method. With no arguments, it will add the Natural Earth ``1:110,000,000`` scale coastline data to the map.

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.coastlines()

Projection classes have options we can use to customize the map

In [ ]:
ccrs.PlateCarree?

In [ ]:
fig, ax = plt.subplots(
    subplot_kw={"projection": ccrs.PlateCarree(central_longitude=180)}
)
ax.coastlines()

### <a id='toc1_3_3_'></a>[Useful methods of a GeoAxes](#toc0_)

The [cartopy.mpl.geoaxes.GeoAxes](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes) class adds a number of useful methods.

Let's take a look at:

 * [set_global](https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.mpl.geoaxes.GeoAxes.html#cartopy.mpl.geoaxes.GeoAxes.set_global) - zoom the map out as much as possible
 * [set_extent](https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.mpl.geoaxes.GeoAxes.html#cartopy.mpl.geoaxes.GeoAxes.set_extent) - zoom the map to the given bounding box
 

 * [gridlines](https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.mpl.geoaxes.GeoAxes.html#cartopy.mpl.geoaxes.GeoAxes.gridlines) - add a graticule (and optionally labels) to the axes
 * [coastlines](https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.mpl.geoaxes.GeoAxes.html#cartopy.mpl.geoaxes.GeoAxes.coastlines) - add Natural Earth coastlines to the axes
 * [stock_img](https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.mpl.geoaxes.GeoAxes.html#cartopy.mpl.geoaxes.GeoAxes.stock_img) - add a low-resolution Natural Earth background image to the axes
 
 
 * [imshow](https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.mpl.geoaxes.GeoAxes.html#cartopy.mpl.geoaxes.GeoAxes.imshow) - add an image (numpy array) to the axes
 * [add_geometries](https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.mpl.geoaxes.GeoAxes.html#cartopy.mpl.geoaxes.GeoAxes.add_geometries) - add a collection of geometries (Shapely / GeoPandas) to the axes
 
### <a id='toc1_3_4_'></a>[Some More Examples of Different Global Projections](#toc0_)

In [ ]:
projections = [
    ccrs.PlateCarree(),
    ccrs.Robinson(),
    ccrs.Mercator(),
    ccrs.Orthographic(),
    ccrs.InterruptedGoodeHomolosine(),
]


for proj in projections:
    fig, ax = plt.subplots(subplot_kw={"projection": proj})
    ax.stock_img()
    ax.coastlines()
    ax.set_title(f"{type(proj)}")

### <a id='toc1_3_5_'></a>[Regional Maps](#toc0_)

To create a regional map, we use the `set_extent` method of GeoAxis to limit the size of the region.

In [ ]:
ax.set_extent?

In [ ]:
central_lon, central_lat = -10, 45
extent = [-40, 20, 30, 60]
fig, ax = plt.subplots(
    subplot_kw={"projection": ccrs.Orthographic(central_lon, central_lat)}
)
ax.set_extent(extent)
ax.gridlines()
ax.coastlines(resolution="50m")

## <a id='toc1_4_'></a>[Adding features to the map](#toc0_)

To give our map more styles and details, we add `cartopy.feature` objects.
Many useful features are built in. These "default features" are at coarse (110m) resolution.

Name | Description
-----|------------
`cartopy.feature.BORDERS` | Country boundaries
`cartopy.feature.COASTLINE` | Coastline, including major islands
`cartopy.feature.LAKES` | Natural and artificial lakes
`cartopy.feature.LAND` | Land polygons, including major islands
`cartopy.feature.OCEAN` | Ocean polygons
`cartopy.feature.RIVERS` | Single-line drainages, including lake centerlines
`cartopy.feature.STATES` | (limited to the United States at this scale)

Below we illustrate these features in a customized map of North America.

In [ ]:
extent = [-120, -70, 24, 50.5]
central_lon = np.mean(extent[:2])
central_lat = np.mean(extent[2:])

fig, ax = plt.subplots(
    subplot_kw={"projection": ccrs.AlbersEqualArea(central_lon, central_lat)},
    figsize=(12, 6),
)
ax.set_extent(extent)

ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.LAND, edgecolor="black")
ax.add_feature(cartopy.feature.LAKES, edgecolor="black")
ax.add_feature(cartopy.feature.RIVERS)
ax.gridlines()

If we want higher-resolution features, Cartopy can automatically download and create them from the [Natural Earth Data](http://www.naturalearthdata.com/) database or the [GSHHS dataset](https://www.ngdc.noaa.gov/mgg/shorelines/gshhs.html) database.

## <a id='toc1_5_'></a>[Adding vector data to the map](#toc0_)

You will typically have your own data to add to a map, of course.

We can plot specific points:

In [ ]:
proj = ccrs.AlbersEqualArea()

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={"projection": proj})
ax.plot(0, 0, marker="o", color="red", markersize=12, transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_global()
ax.gridlines()

Because our map is a matplotlib axis, we can use all the familiar matplotlib commands to make plots.
By default, the map extent will be adjusted to match the data. We can override this with the `.set_global` or `.set_extent` commands.

Another example to show the difference of adding a transform or not:

In [ ]:
# create some test data
new_york = dict(lon=-74.0060, lat=40.7128)
honolulu = dict(lon=-157.8583, lat=21.3069)
lons = [new_york["lon"], honolulu["lon"]]
lats = [new_york["lat"], honolulu["lat"]]

Key point: **the data also have to be transformed to the projection space**.
This is done via the `transform=` keyword in the plotting method. The argument is another `cartopy.crs` object.
If you don't specify a transform, Cartopy assume that the data is using the same projection as the underlying GeoAxis.

From the [Cartopy Documentation](https://scitools.org.uk/cartopy/docs/latest/tutorials/understanding_transform.html)

> The core concept is that the projection of your axes is independent of the coordinate system your data is defined in. The `projection` argument is used when creating plots and determines the projection of the resulting plot (i.e. what the plot looks like). The `transform` argument to plotting functions tells Cartopy what coordinate system your data are defined in.

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.plot(lons, lats, label="Equirectangular straight line")
ax.plot(lons, lats, label="Great Circle", transform=ccrs.Geodetic())
ax.coastlines()
ax.legend()
ax.set_global()

## <a id='toc1_6_'></a>[Adding raster data to the map](#toc0_)

The same principles apply to 2D data. Below we create some example data defined in regular lat / lon coordinates.

In [ ]:
lon = np.linspace(-80, 80, 25)
lat = np.linspace(30, 70, 25)
lon2d, lat2d = np.meshgrid(lon, lat)
data = np.cos(np.deg2rad(lat2d) * 4) + np.sin(np.deg2rad(lon2d) * 4)
plt.contourf(lon2d, lat2d, data)

Now we create a `PlateCarree` projection and plot the data on it without any `transform` keyword.
This happens to work because `PlateCarree` is the simplest projection of lat / lon data.

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.set_global()
ax.coastlines()
ax.contourf(lon, lat, data)

However, if we try the same thing with a different projection, we get the wrong result.

In [ ]:
projection = ccrs.RotatedPole(pole_longitude=-177.5, pole_latitude=37.5)
fig, ax = plt.subplots(subplot_kw={"projection": projection})
ax.set_global()
ax.coastlines()
ax.contourf(lon, lat, data)

To fix this, we need to pass the correct transform argument to `contourf`:

In [ ]:
projection = ccrs.RotatedPole(pole_longitude=-177.5, pole_latitude=37.5)
fig, ax = plt.subplots(subplot_kw={"projection": projection})
ax.set_global()
ax.coastlines()
ax.contourf(lon, lat, data, transform=ccrs.PlateCarree())

## <a id='toc1_7_'></a>[Xarray Integration](#toc0_)

Cartopy transforms can be passed to xarray! This creates a very quick path for creating professional looking maps from netCDF data.

In [ ]:
file_path_data_nc = (
    f"{data_dir}/20190101120000-REMSS-L4_GHRSST-SSTfnd-MW_IR_OI-GLOB-v02.0-fv05.0.nc"
)

In [ ]:
ds = xr.open_dataset(file_path_data_nc)
ds

In [ ]:
sst = ds["analysed_sst"]

fig, ax = plt.subplots(subplot_kw={"projection": ccrs.Robinson()}, figsize=(9, 6))
ax.coastlines()
ax.gridlines()
sst.plot(
    ax=ax, transform=ccrs.PlateCarree(), cbar_kwargs={"shrink": 0.4}, cmap="RdBu_r"
);

## <a id='toc1_8_'></a>[Doing More](#toc0_)

Browse the [Cartopy Gallery](https://scitools.org.uk/cartopy/docs/latest/gallery/index.html) to learn about all the different types of data and plotting methods available!